## импорты

In [ ]:
from datetime import datetime
import requests
import json
import pandas as pd
import nltk
import logging
import json
import re
import requests
from collections import Counter
import logging
from pymystem3 import Mystem
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

# загружаем  и предобрабатываем данные

In [ ]:
def load_stop_words():
    """
    Загружает список стоп-слов из CSV и nltk.
    Если CSV не найден, используется только nltk.
    """
    try:
        stop_words_csv = pd.read_csv('src/stop_words.csv')['word'].to_list()
    except Exception:
        stop_words_csv = []
    stop_words_nltk = nltk.corpus.stopwords.words("russian")
    return set(stop_words_csv + stop_words_nltk)

def split_reviews(text: str):
    """
    Разбивает входной текст на список отзывов по переводам строки.
    Отфильтровывает пустые строки.
    """
    reviews = [line.strip() for line in text.splitlines() if line.strip()]
    if not reviews:
        raise ValueError("Не удалось выделить ни одного отзыва из входного текста")
    return reviews

def text_preproc(text: str, token_pattern: str = r'(?:не\ |ни\ |нет\ |\b)[Ё-ё]{4,}', stop_words=None) -> str:
    """
    выполняет предобработку текста:
    - лемматизация с использованием Mystem
    - поиск токенов по заданному регулярному выражению
    - замена пробелов в найденных токенах на символы подчеркивания
    - исключение токенов, оканчивающихся на стоп-слова

    args:
        text (str): исходный текст для обработки
        token_pattern (str, optional): регулярное выражение для поиска токенов. по умолчанию
            r'(?:не\ |ни\ |нет\ |\b)[Ё-ё]{4,}'.
        stop_words (set, optional): множество стоп-слов для фильтрации токенов

    returns:
        str: обработанный текст, состоящий из отобранных токенов
    """
    if not stop_words:
        stop_words = set()
    try:
        text = ''.join(mystem.lemmatize(text))
        # text = text
        tokens = re.findall(token_pattern, text)
        tokens = [t.replace(' ', '_') for t in tokens
                if t.split()[-1] not in stop_words]
        text = ' '.join(tokens)
        return text
    except:
        return text

## вариант исходный

In [ ]:
def compute_bertopic_coherence_values(docs, limit, start=2, step=3):
    """
    вычисляет значения когерентности для моделей BERTopic, обученных с различными размерами минимального топика

    для каждого значения минимального размера топика:
    - создается и обучается модель BERTopic
    - выполняется предобработка документов
    - строится словарь и корпус для оценки когерентности
    - вычисляется когерентность для топиков модели

    args:
        docs (list of str): список документов для тематического моделирования
        limit (int): верхняя граница для изменения минимального размера топика
        start (int, optional): начальное значение минимального размера топика. По умолчанию 2
        step (int, optional): шаг изменения минимального размера топика. По умолчанию 3

    returns:
        tuple:
            - topic_models (list): список обученных моделей BERTopic
            - coherence_values (list): список значений когерентности, соответствующих моделям
    """
    coherence_values = []
    topic_models = []
    for size in range(start, limit, step):
        topic_model = BERTopic(
            min_topic_size=size,
            language="russian",
            # n_gram_range=(2, 3)
        )
        topics, _ = topic_model.fit_transform(docs)

        cleaned_docs = topic_model._preprocess_text(docs)
        vectorizer = topic_model.vectorizer_model
        analyzer = vectorizer.build_analyzer()
        tokens = [analyzer(doc) for doc in cleaned_docs]

        dictionary = corpora.Dictionary(tokens)
        corpus = [dictionary.doc2bow(token) for token in tokens]

        topics = topic_model.get_topics()
        topics.pop(-1, None)
        topic_words = [
            [word for word, _ in topic_model.get_topic(topic)] for topic in topics
        ]
        if not topic_words:
            continue

        coherence_model = CoherenceModel(
            topics=topic_words,
            texts=tokens,
            corpus=corpus,
            dictionary=dictionary,
            coherence="c_v",
        )
        coherence_values.append(coherence_model.get_coherence())
        topic_models.append(topic_model)

    return topic_models, coherence_values
    
def get_representative_texts(lemmatized, originals):
    """
    извлекает представительные тексты из оригинальных документов на основе их лемматизированных версий
    и тематического моделирования с использованием BERTopic

    если количество лемматизированных текстов меньше или равно 10, возвращаются все оригинальные тексты
    В противном случае:
    - выбирается оптимальная модель BERTopic на основе максимального значения когерентности
    - извлекаются тексты, присутствующие в представительных документах каждого топика

    args:
        lemmatized (list of str): список лемматизированных текстов
        originals (list of str): список оригинальных текстов

    returns:
        list of str: список представительных оригинальных текстов
    """
    print('get_representative_texts')
    if len(lemmatized) <= 10:
        return originals

    topic_models, coherence_values = compute_bertopic_coherence_values(
        lemmatized, 5, start=2, step=1
    )
    if not len(coherence_values):
        return originals

    max_value = max(coherence_values)
    max_index = coherence_values.index(max_value)
    optimal_model = topic_models[max_index]
    model_topics = optimal_model.get_topic_info()

    res = []

    for lem, orig in zip(lemmatized, originals):
        for topic_index, topic_row in model_topics.iterrows():
            representative_docs = topic_row["Representative_Docs"]
            if lem in representative_docs:
                res.append(orig)

    return res

In [ ]:
def get_summary(text):
    try:
        reviews = split_reviews(text)
        print('размер исходного датасета', len(reviews))
        stop_words = load_stop_words()

        # получаем лемматизированную версию каждого отзыва
        lemmas = [text_preproc(review, stop_words=stop_words) for review in reviews]

        # пытаемся выделить репрезентативные отзывы (при небольшом числе отзывов функция вернёт исходный список)
        try:
            rep_reviews = get_representative_texts(lemmas, reviews)
            print('размер обработанного датасета', len(rep_reviews))
        except Exception as err:
            # если не удалось получить репрезентативные отзывы — берём первые 5 отзывов
            rep_reviews = reviews[:5]

        # summary = get_summary(rep_reviews) # пока закомментируем, для теста не нужно

    except Exception as e:
        log_request(endpoint="summarize", status="error")
        raise HTTPException(status_code=500, detail=str(e))

    return rep_reviews

In [ ]:
text = ""

In [ ]:
%%time
summary = get_summary(text)

## вариант 2 с небольшим ускорением

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora
from gensim.models import CoherenceModel
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
from umap import UMAP
import numpy as np
import pandas as pd
import logging

# загружаем эмбеддинг-модель один раз (использует GPU)
embedding_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cuda"
)

# общий CountVectorizer для всех запусков
vectorizer_model = CountVectorizer()

# кеш для эмбеддингов, чтобы не пересчитывать при одинаковом входе
embedding_cache = {}

In [ ]:
def compute_bertopic_coherence_values(docs, embeddings, dictionary, tokens, corpus, limit, start=2, step=1):
    coherence_values = []
    topic_models = []

    for size in range(start, limit, step):
        topic_model = BERTopic(
            min_topic_size=size,
            language="russian",
            calculate_probabilities=False,  # выключаем soft clustering (ускоряет)
            nr_topics=10,
            vectorizer_model=vectorizer_model,
            umap_model=UMAP(n_components=5, metric='cosine', low_memory=True),
            hdbscan_model=HDBSCAN(min_cluster_size=size)
        )

        topics, _ = topic_model.fit_transform(docs, embeddings)

        model_topics = topic_model.get_topics()
        model_topics.pop(-1, None)  # удаляем outlier (-1), если есть

        topic_words = [
            [word for word, _ in topic_model.get_topic(topic)]
            for topic in model_topics
        ]
        if not topic_words:
            continue

        coherence_model = CoherenceModel(
            topics=topic_words,
            texts=tokens,
            corpus=corpus,
            dictionary=dictionary,
            coherence="c_v",
            processes=4  # многопроцессный расчёт
        )
        coherence_values.append(coherence_model.get_coherence())
        topic_models.append(topic_model)

    return topic_models, coherence_values

In [ ]:
# вариант с 1м документов в теме
def get_representative_texts(lemmatized, originals):
    logging.info("get_representative_texts")

    
    if isinstance(lemmatized, pd.Series):
        lemmatized = lemmatized.reset_index(drop=True)
    if isinstance(originals, pd.Series):
        originals = originals.reset_index(drop=True)

    
    if len(lemmatized) <= 10:
        return list(originals)

    try:
        # Проверяем кеш по хэшу объединённого текста
        joined_text = "\n".join(lemmatized)
        if joined_text in embedding_cache:
            embeddings = embedding_cache[joined_text]
        else:
            embeddings = embedding_model.encode(
                lemmatized, show_progress_bar=False, batch_size=64
            )
            embedding_cache[joined_text] = embeddings

        # Токенизация и подготовка корпуса для Gensim
        analyzer = vectorizer_model.build_analyzer()
        tokens = [analyzer(doc) for doc in lemmatized]
        dictionary = corpora.Dictionary(tokens)
        corpus = [dictionary.doc2bow(t) for t in tokens]

        # Обучаем несколько моделей и выбираем лучшую
        topic_models, coherence_values = compute_bertopic_coherence_values(
            lemmatized, embeddings, dictionary, tokens, corpus,
            limit=5, start=2, step=1
        )

        if not coherence_values:
            return list(originals)

        max_index = coherence_values.index(max(coherence_values))
        optimal_model = topic_models[max_index]

        # Получаем информацию о документах и метках тем
        document_info = optimal_model.get_document_info()
        topics = document_info["Topic"].to_numpy()

        res = []

        # Для каждой темы находим наиболее "репрезентативный" документ (ближайший к центру)
        for topic_id in np.unique(topics):
            if topic_id == -1:
                continue  # пропускаем выбросы
            doc_indices = np.where(topics == topic_id)[0]
            if len(doc_indices) == 0:
                continue
            cluster_embeddings = embeddings[doc_indices]
            centroid = cluster_embeddings.mean(axis=0)
            dists = np.linalg.norm(cluster_embeddings - centroid, axis=1)
            best_doc_idx = doc_indices[np.argmin(dists)]

            # Безопасно достаём текст по позиции
            if isinstance(originals, pd.Series):
                res.append(originals.iloc[best_doc_idx])
            else:
                res.append(originals[best_doc_idx])

        return res

    except Exception as err:
        logging.warning("Couldn't get topics")
        logging.exception(err)
        return list(originals)[:5]

In [ ]:
%%time
summary = get_summary(text)

## вариант 3 - итоговый 

In [ ]:
from cuml.manifold import UMAP as GPU_UMAP
from cuml.cluster import HDBSCAN as GPU_HDBSCAN

def compute_bertopic_coherence_values(docs, embeddings, dictionary, tokens, corpus, limit, start=2, step=1):
    coherence_values = []
    topic_models = []

    for size in range(start, limit, step):
        # на GPU модели
        umap_model = GPU_UMAP(n_components=5, n_neighbors=15, min_dist=0.0, metric="cosine")
        hdbscan_model = GPU_HDBSCAN(min_cluster_size=size)

        topic_model = BERTopic(
            min_topic_size=size,
            language="russian",
            calculate_probabilities=False,
            nr_topics=10,
            vectorizer_model=vectorizer_model,
            umap_model=umap_model,
            hdbscan_model=hdbscan_model
        )

        topics, _ = topic_model.fit_transform(docs, embeddings)


        model_topics = {
            topic: words for topic, words in topic_model.get_topics().items()
            if words and topic != -1
        }
        
        topic_words = [
            [word for word, _ in model_topics[topic]]
            for topic in model_topics
        ]
        
        if not topic_words:
            continue  # пропускаем модель без тем


        coherence_model = CoherenceModel(
            topics=topic_words,
            texts=tokens,
            corpus=corpus,
            dictionary=dictionary,
            coherence="c_v",
            processes=4
        )
        coherence_values.append(coherence_model.get_coherence())
        topic_models.append(topic_model)

    return topic_models, coherence_values


In [ ]:
def get_representative_texts(lemmatized, originals, mode='strict'):
    """
    Возвращает список репрезентативных оригинальных отзывов.
    - mode='strict': по 1 документу на каждую тему (ближайший к центроиду).
    - mode='expanded': все документы из BERTopic -> Representative_Docs.
    """

    logging.info("get_representative_texts")

    if isinstance(lemmatized, pd.Series):
        lemmatized = lemmatized.reset_index(drop=True)
    if isinstance(originals, pd.Series):
        originals = originals.reset_index(drop=True)

    if len(lemmatized) <= 10:
        return list(originals)

    try:
        joined_text = "\n".join(lemmatized)
        if joined_text in embedding_cache:
            embeddings = embedding_cache[joined_text]
        else:
            embeddings = embedding_model.encode(
                lemmatized, show_progress_bar=False, batch_size=64
            )
            embedding_cache[joined_text] = embeddings

        analyzer = vectorizer_model.build_analyzer()
        tokens = [analyzer(doc) for doc in lemmatized]
        dictionary = corpora.Dictionary(tokens)
        corpus = [dictionary.doc2bow(t) for t in tokens]

        topic_models, coherence_values = compute_bertopic_coherence_values(
            lemmatized, embeddings, dictionary, tokens, corpus,
            limit=5, start=2, step=1
        )

        if not coherence_values:
            return list(originals)

        max_index = coherence_values.index(max(coherence_values))
        optimal_model = topic_models[max_index]
        document_info = optimal_model.get_document_info(lemmatized)
        topics = document_info["Topic"].to_numpy()

        res = []

        if mode == 'strict':
            # строгий режим: по 1 документу на тему
            for topic_id in np.unique(topics):
                if topic_id == -1:
                    continue
                doc_indices = np.where(topics == topic_id)[0]
                if len(doc_indices) == 0:
                    continue
                cluster_embeddings = embeddings[doc_indices]
                centroid = cluster_embeddings.mean(axis=0)
                dists = np.linalg.norm(cluster_embeddings - centroid, axis=1)
                best_doc_idx = doc_indices[np.argmin(dists)]
                if isinstance(originals, pd.Series):
                    res.append(originals.iloc[best_doc_idx])
                else:
                    res.append(originals[best_doc_idx])

        elif mode == 'expanded':
            # расширенный режим: Representative_Docs от BERTopic
            rep_docs_set = set()
            topic_info = optimal_model.get_topic_info()
            for _, row in topic_info.iterrows():
                if row["Topic"] == -1:
                    continue
                rep_docs_set.update(row["Representative_Docs"])

            # сопоставляем лемматизированные с оригинальными
            for lem, orig in zip(lemmatized, originals):
                if lem in rep_docs_set:
                    res.append(orig)

        else:
            raise ValueError("mode должен быть 'strict' или 'expanded'")

        return res

    except Exception as err:
        logging.warning("Couldn't get topics")
        logging.exception(err)
        return list(originals)[:5]


In [ ]:
def get_summary(text):
    try:
        reviews = split_reviews(text)
        print('размер исходного датасета', len(reviews))
        stop_words = load_stop_words()

        # получаем лемматизированную версию каждого отзыва
        lemmas = [text_preproc(review, stop_words=stop_words) for review in reviews]

        # пытаемся выделить репрезентативные отзывы (при небольшом числе отзывов функция вернёт исходный список)
        try:
            rep_reviews = get_representative_texts(lemmas, reviews,
                                            mode='expanded'
                                             ) # ВАЖНО параметр mode отвечает за то, сколько текстов выберем 1 или все из репр докс)
            print('размер обработанного датасета', len(rep_reviews))
        except Exception as err:
            # если не удалось получить репрезентативные отзывы — берём первые 5 отзывов
            rep_reviews = reviews[:5]

        # summary = get_summary(rep_reviews) # пока закомментируем, для теста не нужно

    except Exception as e:
        log_request(endpoint="summarize", status="error")
        raise HTTPException(status_code=500, detail=str(e))

    return rep_reviews

In [ ]:
%%time
summary = get_summary(text)